In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [58]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from PIL import Image


In [59]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data loading and preprocessing
def load_data():
    data_dir = '/content/drive/MyDrive/JPEGImages'
    transform_train = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load dataset without applying any transforms
    full_dataset = ImageFolder(data_dir)

    # Splitting dataset into train and test
    train_indices, test_indices = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=42)

    # Apply different transforms for train and test datasets
    train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
    test_dataset = torch.utils.data.Subset(full_dataset, test_indices)
    train_dataset.dataset.transform = transform_train
    test_dataset.dataset.transform = transform_test

    # Calculate class weights for handling imbalance
    class_counts = torch.bincount(torch.tensor(full_dataset.targets), minlength=50)
    class_weights = 1.0 / (class_counts.float() + 1e-6)
    sample_weights = class_weights[full_dataset.targets]
    train_sample_weights = sample_weights[train_indices]

    # Create samplers
    train_sampler = WeightedRandomSampler(train_sample_weights, len(train_indices), replacement=True)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

    return train_loader, test_loader


In [60]:
# Model definition
def initialize_model(num_classes=50):
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_ftrs, num_classes),
        nn.BatchNorm1d(num_classes)
    )
    model.to(device)
    return model

In [61]:
# Training the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    scaler = torch.cuda.amp.GradScaler()  # For mixed precision training
    for epoch in range(num_epochs):
        start_time = time.time()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass with mixed precision
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if i % 20 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

        epoch_duration = time.time() - start_time
        print(f"Epoch {epoch+1} completed in {epoch_duration:.2f} seconds")


In [62]:
# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')





In [63]:
# Main
def main():
    train_loader, test_loader = load_data()
    model = initialize_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_model(model, train_loader, criterion, optimizer)
    evaluate_model(model, test_loader)

if __name__ == '__main__':
    main()

Epoch [1/10], Step [1/129], Loss: 4.2647
Epoch [1/10], Step [21/129], Loss: 2.1551
Epoch [1/10], Step [41/129], Loss: 1.9102
Epoch [1/10], Step [61/129], Loss: 1.8914
Epoch [1/10], Step [81/129], Loss: 1.6256
Epoch [1/10], Step [101/129], Loss: 1.6691
Epoch [1/10], Step [121/129], Loss: 1.4305
Epoch 1 completed in 29.42 seconds
Epoch [2/10], Step [1/129], Loss: 1.5404
Epoch [2/10], Step [21/129], Loss: 1.4630
Epoch [2/10], Step [41/129], Loss: 1.3036
Epoch [2/10], Step [61/129], Loss: 1.5324
Epoch [2/10], Step [81/129], Loss: 1.2904
Epoch [2/10], Step [101/129], Loss: 0.8860
Epoch [2/10], Step [121/129], Loss: 1.1138
Epoch 2 completed in 28.69 seconds
Epoch [3/10], Step [1/129], Loss: 1.0593
Epoch [3/10], Step [21/129], Loss: 0.9275
Epoch [3/10], Step [41/129], Loss: 1.0424
Epoch [3/10], Step [61/129], Loss: 1.2023
Epoch [3/10], Step [81/129], Loss: 0.9401
Epoch [3/10], Step [101/129], Loss: 0.7538
Epoch [3/10], Step [121/129], Loss: 0.9458
Epoch 3 completed in 28.80 seconds
Epoch [4/1